## Imports

In [2]:
#Commonted out the below lines as the packages are already installed in the environment

#!pip install timm
#!pip install nibabel


import nibabel as nib
import timm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



c:\Users\Gelo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [4]:
# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:

# Specify the path to the NII file
file_path = 'sub-0010001_ses-1_run-1_T1w.nii.gz'

# Load the NII image
image = nib.load(file_path)

# Access the image data
data = image.get_fdata()

# Print the shape of the image data
print(data.shape)

(128, 256, 256)
